In [ ]:
class Vertex:
    def __init__(self):
        self._links = []

    def get_links(self):
        return self._links

    links = property(get_links)


class Link:
    def __init__(self, v1: Vertex, v2: Vertex, dist=1):
        self._v1 = v1
        self._v2 = v2
        self._dist = dist

    def get_v1(self):
        return self._v1

    def get_v2(self):
        return self._v2

    def get_dist(self):
        return self._dist

    v1 = property(get_v1)
    v2 = property(get_v2)
    dist = property(get_dist)


class LinkedGraph:
    def __init__(self, links=None, vertex=None):
        if links is None:
            self._links = []
        else:
            self._links = links
        if vertex is None:
            self._vertex = []
        else:
            self._vertex = vertex

    def add_vertex(self, v: Vertex):
        if v not in self._vertex:
            self._vertex.append(v)

    def add_link(self, link: Link):
        if link not in self._links:
            same_link_exists = any(l for l in self._links if (l.get_v1() == link.get_v1() and
                                   l.get_v2() == link.get_v2()) or
                                   (l.get_v1() == link.get_v2() and
                                   l.get_v2() == link.get_v1()))
            if not same_link_exists:
                self._links.append(link)
                link.v1.get_links().append(link.v2)
                link.v2.get_links().append(link.v1)
                self.add_vertex(link.v1)
                self.add_vertex(link.v2)

    def find_path(self, start_v: Vertex, stop_v: Vertex):
        processed = set()
        shortest_dists = {x: float('inf') for x in self._vertex}
        shortest_dists[start_v] = 0

        def check_vertex(vertex):
            if vertex == stop_v:
                return
            unprocessed = list(filter(lambda x: x not in processed, vertex.get_links()))
            for neighbour in unprocessed:
                for link in self._links:
                    if (link.v1 == vertex and link.v2 == neighbour) or (link.v1 == neighbour and link.v2 == vertex):
                        if shortest_dists[vertex] + link.get_dist() < shortest_dists[neighbour]:
                            shortest_dists[neighbour] = shortest_dists[vertex] + link.get_dist()
                            break
            processed.add(vertex)
            for neighbour in unprocessed:
                check_vertex(neighbour)

        check_vertex(start_v)
        processed.clear()
        route_back = [stop_v]

        def path(vertex: Vertex):
            if vertex == start_v:
                route = route_back[::-1]
                route_links = []
                for i in range(len(route) - 1):
                    for link in self._links:
                        if (link.v1 == route[i] and link.v2 == route[i + 1]) or\
                                (link.v1 == route[i + 1] and link.v2 == route[i]):
                            route_links.append(link)
                return route, route_links

            unprocessed = list(filter(lambda x: x not in processed, vertex.get_links()))
            for neighbour in unprocessed:
                for link in self._links:
                    if (link.v1 == vertex and link.v2 == neighbour) or (link.v1 == neighbour and link.v2 == vertex):
                        if shortest_dists[vertex] - link.get_dist() == shortest_dists[neighbour]:
                            route_back.append(neighbour)
                            break
            processed.add(vertex)
            return path(route_back[-1])

        return path(stop_v)


class Station(Vertex):
    def __init__(self, name):
        super().__init__()
        self._name = name

    def __str__(self):
        return self._name

    def __repr__(self):
        return self._name


class LinkMetro(Link):
    def __init__(self, v1, v2, dist):
        super().__init__(v1, v2, dist)

In [ ]:
map2 = LinkedGraph()
v1 = Vertex()
v2 = Vertex()
v3 = Vertex()
v4 = Vertex()
v5 = Vertex()
map2.add_link(Link(v1, v2))
map2.add_link(Link(v2, v3))
map2.add_link(Link(v2, v4))
map2.add_link(Link(v3, v4))
map2.add_link(Link(v4, v5))
assert len(map2._links) == 5, "неверное число связей в списке _links класса LinkedGraph"
assert len(map2._vertex) == 5, "неверное число вершин в списке _vertex класса LinkedGraph"
map2.add_link(Link(v2, v1))
assert len(map2._links) == 5, "метод add_link() добавил связь Link(v2, v1), хотя уже имеется связь Link(v1, v2)"
path = map2.find_path(v1, v5)
s = sum([x.dist for x in path[1]])
assert s == 3, "неверная суммарная длина маршрута, возможно, некорректно работает объект-свойство dist"
assert issubclass(Station, Vertex) and issubclass(LinkMetro, Link), "класс Station должен наследоваться от класса Vertex, а класс LinkMetro от класса Link"
map2 = LinkedGraph()
v1 = Station("1")
v2 = Station("2")
v3 = Station("3")
v4 = Station("4")
v5 = Station("5")
map2.add_link(LinkMetro(v1, v2, 1))
map2.add_link(LinkMetro(v2, v3, 2))
map2.add_link(LinkMetro(v2, v4, 7))
map2.add_link(LinkMetro(v3, v4, 3))
map2.add_link(LinkMetro(v4, v5, 1))
assert len(map2._links) == 5, "неверное число связей в списке _links класса LinkedGraph"
assert len(map2._vertex) == 5, "неверное число вершин в списке _vertex класса LinkedGraph"
path = map2.find_path(v1, v5)
assert str(path[0]) == '[1, 2, 3, 4, 5]', path[0]
s = sum([x.dist for x in path[1]])
assert s == 7, "неверная суммарная длина маршрута для карты метро"